# Hypothesis
- Gait is a more accurate measurement for determining weight and BMR than height.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.api import Holt
from fbprophet import Prophet

from sklearn.model_selection import TimeSeriesSplit
from sklearn import metrics

import math

In [2]:
df = pd.read_csv('Total_Fitbit_2.csv')

In [3]:
df['Date'] = pd.to_datetime(df.Date)

In [4]:
df.set_index('Date', drop=True, inplace= True)

In [5]:
df.replace(',','', regex=True, inplace=True)

In [6]:
cols = ['Calories_Burned', 'Steps', 'Distance', 'Floors', 'Minutes_Sedentary',
       'Minutes_Lightly_Active', 'Minutes_Fairly_Active',
       'Minutes_Very_Active', 'Activity_Calories', 'Calories_In']

df = df[cols].astype(float)

In [7]:
df.rename(index=str, columns={"Minutes_Sedentary": "Min_S", "Minutes_Lightly_Active": "Min_LA",
                             "Minutes_Fairly_Active": "Min_FA","Minutes_Very_Active": "Min_VA",
                             "Activity_Calories": "Act_Cal","Calories_In": "Cal_In", "Calories_Burned": "Cal_Burn",
                             "Distance":"Dist"}, inplace=True)

In [8]:
df.head()

,Cal_Burn,Steps,Dist,Floors,Min_S,Min_LA,Min_FA,Min_VA,Act_Cal,Cal_In
Date,,,,,,,,,,
2018-04-26 00:00:00,2635.0,2931.0,1.38,2.0,1347.0,93.0,0.0,0.0,566.0,0.0
2018-04-27 00:00:00,3649.0,9971.0,4.70,19.0,1158.0,219.0,46.0,17.0,1752.0,0.0
2018-04-28 00:00:00,3379.0,3521.0,1.66,2.0,702.0,155.0,22.0,11.0,1124.0,0.0
2018-04-29 00:00:00,2851.0,1639.0,0.77,0.0,891.0,122.0,0.0,0.0,602.0,0.0
2018-04-30 00:00:00,2825.0,1323.0,0.62,0.0,880.0,104.0,0.0,0.0,540.0,0.0


In [9]:
df['Dist_in'] = df.Dist*63360

In [10]:
df['Gait'] = df.Dist_in/df.Steps

In [11]:
height_cm = ((df.Gait.mean())/0.43)*2.54
# (the 0.43 comes from -> https://www.scientificamerican.com/article/bring-science-home-estimating-height-walk/)
# gh= gait_height
# ah= actual_height
gh = ((df.Gait.mean())/0.43)*2.54
ah = 182.88
a = 28
b = 2145

In [12]:
weight_lb_actual_height = (((((6.25*ah)-(5*a)+5)-b)/-10)*2.20462)
weight_lb_gait_height = (((((6.25*gh)-(5*a)+5)-b)/-10)*2.20462)
print(weight_lb_actual_height)
print(weight_lb_gait_height)

250.665294
258.6723124224538


In [47]:
df2 = pd.read_csv('Fitabase_Data_4.12.16-5.12.16/dailyActivity_merged.csv')
df3 = pd.read_csv('Fitabase_Data_4.12.16-5.12.16/dailyCalories_merged.csv') #Not Needed
df4 = pd.read_csv('Fitabase_Data_4.12.16-5.12.16/dailyIntensities_merged.csv') #Not Needed
df5 = pd.read_csv('Fitabase_Data_4.12.16-5.12.16/dailySteps_merged.csv') #Not Needed
df6 = pd.read_csv('Fitabase_Data_4.12.16-5.12.16/heartrate_seconds_merged.csv')
df7 = pd.read_csv('Fitabase_Data_4.12.16-5.12.16/hourlyCalories_merged.csv')
df8 = pd.read_csv('Fitabase_Data_4.12.16-5.12.16/hourlyIntensities_merged.csv')
df9 = pd.read_csv('Fitabase_Data_4.12.16-5.12.16/hourlySteps_merged.csv')
df10 = pd.read_csv('Fitabase_Data_4.12.16-5.12.16/minuteCaloriesNarrow_merged.csv')
df11 = pd.read_csv('Fitabase_Data_4.12.16-5.12.16/minuteCaloriesWide_merged.csv')
df12 = pd.read_csv('Fitabase_Data_4.12.16-5.12.16/minuteIntensitiesNarrow_merged.csv') #Not Needed
df13 = pd.read_csv('Fitabase_Data_4.12.16-5.12.16/minuteIntensitiesWide_merged.csv') #Not Needed
df14 =  pd.read_csv('Fitabase_Data_4.12.16-5.12.16/minuteMETsNarrow_merged.csv')
df15 =  pd.read_csv('Fitabase_Data_4.12.16-5.12.16/minuteSleep_merged.csv')
df16 =  pd.read_csv('Fitabase_Data_4.12.16-5.12.16/minuteStepsNarrow_merged.csv') #Not Needed
df17 =  pd.read_csv('Fitabase_Data_4.12.16-5.12.16/minuteStepsWide_merged.csv')
df18 =  pd.read_csv('Fitabase_Data_4.12.16-5.12.16/sleepDay_merged.csv')
df19 =  pd.read_csv('Fitabase_Data_4.12.16-5.12.16/weightLogInfo_merged.csv')

In [84]:
df2.head(7).T

,0,1,2,3,4,5,6
Id,1503960366,1503960366,1503960366,1503960366,1503960366,1503960366,1503960366
ActivityDate,4/12/2016,4/13/2016,4/14/2016,4/15/2016,4/16/2016,4/17/2016,4/18/2016
TotalSteps,13162,10735,10460,9762,12669,9705,13019
TotalDistance,8.5,6.97,6.74,6.28,8.16,6.48,8.59
TrackerDistance,8.5,6.97,6.74,6.28,8.16,6.48,8.59
LoggedActivitiesDistance,0,0,0,0,0,0,0
VeryActiveDistance,1.88,1.57,2.44,2.14,2.71,3.19,3.25
ModeratelyActiveDistance,0.55,0.69,0.4,1.26,0.41,0.78,0.64
LightActiveDistance,6.06,4.71,3.91,2.83,5.04,2.51,4.71
SedentaryActiveDistance,0,0,0,0,0,0,0


In [33]:
shape = df2.shape
participants = len(df2.Id.unique())
numberofdays = len(df2.ActivityDate.unique())
print(f'Shape of the data set is {shape}')
print(f'The number of participants is {participants}')
print(f'The number of days is {numberofdays}')

Shape of the data set is (940, 15)
The number of participants is 33
The number of days is 31


In [69]:
df2.Id.unique()

array([1503960366, 1624580081, 1644430081, 1844505072, 1927972279,
       2022484408, 2026352035, 2320127002, 2347167796, 2873212765,
       3372868164, 3977333714, 4020332650, 4057192912, 4319703577,
       4388161847, 4445114986, 4558609924, 4702921684, 5553957443,
       5577150313, 6117666160, 6290855005, 6775888955, 6962181067,
       7007744171, 7086361926, 8053475328, 8253242879, 8378563200,
       8583815059, 8792009665, 8877689391])

In [83]:
df19.head()

,Id,Date,WeightKg,WeightPounds,Fat,BMI,IsManualReport,LogId
0,1503960366,5/2/2016 11:59:59 PM,52.599998,115.963147,22.0,22.650000,True,1462233599000
1,1503960366,5/3/2016 11:59:59 PM,52.599998,115.963147,NaN,22.650000,True,1462319999000
2,1927972279,4/13/2016 1:08:52 AM,133.500000,294.317120,NaN,47.540001,False,1460509732000
3,2873212765,4/21/2016 11:59:59 PM,56.700001,125.002104,NaN,21.450001,True,1461283199000
4,2873212765,5/12/2016 11:59:59 PM,57.299999,126.324875,NaN,21.690001,True,1463097599000


In [67]:
df19.Id.unique()

array([1503960366, 1927972279, 2873212765, 4319703577, 4558609924,
       5577150313, 6962181067, 8877689391])

For Gait BMR calculations the following dataframes are needed
df2, df7, and df19

In [78]:
df7.Calories.where(df7.Id==1503960366).sum()/31

1815.7096774193549